In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import AmazonGoogleBenchmark

benchmark = AmazonGoogleBenchmark(data_dir_path="../data/")
benchmark

22:10:26 INFO:Extracting Amazon-Google...
22:10:26 INFO:Reading Amazon-Google record_dict...
22:10:26 INFO:Reading Amazon-Google train.csv...
22:10:26 INFO:Reading Amazon-Google valid.csv...
22:10:26 INFO:Reading Amazon-Google test.csv...


<AmazonGoogleBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Amazon-Google/amazon_google_exp_data.zip

## Preprocess

In [5]:
field_list = ['title', 'manufacturer', 'price']

In [6]:
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for record in benchmark.record_dict.values():
    for field in field_list:
        record[field] = clean_str(record[field])

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
from entity_embed.data_utils.field_config_parser import DEFAULT_ALPHABET

alphabet = DEFAULT_ALPHABET
field_config_dict = {
    'title': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'title_semantic': {
        'key': 'title',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
    },
    'manufacturer': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'price': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    }
}

In [9]:
from entity_embed import FieldConfigDictParser

record_numericalizer = FieldConfigDictParser.from_dict(
    field_config_dict, record_list=benchmark.record_dict.values())

22:10:26 INFO:For field=title, computing actual max_str_len
22:10:26 INFO:For field=title, using actual_max_str_len=26
22:10:26 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
22:10:30 INFO:For field=manufacturer, computing actual max_str_len
22:10:30 INFO:actual_max_str_len=15 must be even to enable NN pooling. Updating to 16
22:10:30 INFO:For field=manufacturer, using actual_max_str_len=16
22:10:30 INFO:For field=price, computing actual max_str_len
22:10:30 INFO:actual_max_str_len=9 must be even to enable NN pooling. Updating to 10
22:10:30 INFO:For field=price, using actual_max_str_len=10


In [10]:
batch_size = 32
eval_batch_size = 128
datamodule = benchmark.build_datamodule(
    record_numericalizer=record_numericalizer,
    batch_size=batch_size,
    eval_batch_size=eval_batch_size,
    random_seed=random_seed
)

## Training

In [11]:
from entity_embed import LinkageEmbed
from pytorch_metric_learning.losses import NTXentLoss
from pytorch_metric_learning.reducers import AvgNonZeroReducer

ann_k = 100
model = LinkageEmbed(
    record_numericalizer,
    ann_k=ann_k,
    source_field=datamodule.source_field,
    left_source=datamodule.left_source,
    sim_threshold_list=[0.5, 0.7, 0.9],
)

In [12]:
trainer = model.fit(
    datamodule,
    min_epochs=5,
    max_epochs=100,
    check_val_every_n_epoch=1,
    early_stop_monitor="valid_f1_at_0.7",
    tb_save_dir='../tb_logs',
    tb_name=f'matcher-{benchmark.dataset_name}'
)

22:10:30 INFO:GPU available: True, used: True
22:10:30 INFO:TPU available: False, using: 0 TPU cores
22:10:30 INFO:Train positive pair count: 699
22:10:30 INFO:Train negative pair count: 6175
22:10:30 INFO:Valid positive pair count: 234
22:10:30 INFO:Valid positive pair count: 2059
22:10:30 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
22:10:45 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 7.2 M 
1 | loss_fn     | SupConLoss | 0     
-------------------------------------------
5.5 M     Trainable params
1.7 M     Non-trainable params
7.2 M     Total params
28.776    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/home/fjsj/workspace/entity-embed/venv/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
22:13:14 INFO:Loading the best validation model from ../tb_logs/matcher-Amazon-Google/version_38/checkpoints/epoch=4-step=2589.ckpt...


In [13]:
model.validate(datamodule)

{'valid_f1_at_0.5': 0.09853295379899278,
 'valid_f1_at_0.7': 0.20829015544041452,
 'valid_f1_at_0.9': 0.27109974424552435,
 'valid_pair_entity_ratio_at_0.5': 2.306013837147419,
 'valid_pair_entity_ratio_at_0.7': 0.9026077700904737,
 'valid_pair_entity_ratio_at_0.9': 0.2916444917509313,
 'valid_precision_at_0.5': 0.05192707131317794,
 'valid_precision_at_0.7': 0.11851415094339622,
 'valid_precision_at_0.9': 0.19343065693430658,
 'valid_recall_at_0.5': 0.9615384615384616,
 'valid_recall_at_0.7': 0.8589743589743589,
 'valid_recall_at_0.9': 0.452991452991453}

In [14]:
model.get_pool_weights()

{'title': 0.3568376898765564,
 'title_semantic': 0.3908023536205292,
 'manufacturer': 0.10031930357217789,
 'price': 0.15204067528247833}

## Testing

In [15]:
model.test(datamodule)

22:13:17 INFO:Test positive pair count: 234
22:13:17 INFO:Test positive pair count: 2059


{'test_f1_at_0.5': 0.0948549650645776,
 'test_f1_at_0.7': 0.20512820512820512,
 'test_f1_at_0.9': 0.2911392405063291,
 'test_pair_entity_ratio_at_0.5': 2.412144008597528,
 'test_pair_entity_ratio_at_0.7': 0.9639978506179473,
 'test_pair_entity_ratio_at_0.9': 0.2987641053197206,
 'test_precision_at_0.5': 0.04989975495656048,
 'test_precision_at_0.7': 0.11594202898550725,
 'test_precision_at_0.9': 0.2068345323741007,
 'test_recall_at_0.5': 0.9572649572649573,
 'test_recall_at_0.7': 0.8888888888888888,
 'test_recall_at_0.9': 0.49145299145299143}